In [21]:
import argparse
import logging
import os
import sys
import json


from toolz.curried import keyfilter

from openfisca_parsers import exceptions, loading, module_pass, class_pass, function_pass

In [31]:
import importlib

importlib.reload(loading)
importlib.reload(exceptions)
importlib.reload(module_pass)
importlib.reload(class_pass)
importlib.reload(function_pass)


<module 'openfisca_parsers.function_pass' from '/data/projects/openfisca/openfisca-parsers/openfisca_parsers/function_pass.py'>

# Loading

In [4]:
redbaron_trees = loading.load()

# Module parsing

In [13]:
parsed_modules = module_pass.parse(redbaron_trees)

# Class parsing

In [19]:
parsed_classes = class_pass.parse(parsed_modules)

# Function parsing

In [43]:
importlib.reload(function_pass)
parsed_classes = function_pass.parse(parsed_modules, parsed_classes)

*** Visiting module prelevements_obligatoires/impot_revenu/ir.py
Visiting class rfr_rvcm to parse its function(s)
Visiting function function
Visiting class enfant_majeur_celibataire_sans_enfant to parse its function(s)
Visiting function function
Visiting class indu_plaf_abat_pen to parse its function(s)
Visiting function function
Visiting class revenu_assimile_salaire_apres_abattements to parse its function(s)
Visiting function function
Visiting class ir_brut to parse its function(s)
Visiting function function
Visiting class revenu_activite to parse its function(s)
Visiting function function
Visiting class retraite_titre_onereux_net_declarant1 to parse its function(s)
Visiting class enfant_a_charge to parse its function(s)
Visiting function function
Visiting class cehr to parse its function(s)
Visiting function function
Visiting class ip_net to parse its function(s)
Visiting function function
Visiting class rev_cat to parse its function(s)
Visiting function function
Visiting class micr

NotImplementedParsingError: Unknown self op holder.

# Debugging

In [44]:
exceptions.angry_rbnode

<NameNode path=[76, 'value', 10, 'value', 2, 'value', 'value', 2], "holder" 140127652633736, on AtomtrailersNode 140127652632448>

In [ ]:
parsed_functions_counter / (parsed_functions_counter + len(functions_too_complex))